In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn.model_selection import KFold
from MTS import *
from orthogonal_array import *

In [2]:
# ファイル読み込み
IN_FILE = '../data/SkillCraft1_Dataset.csv'
df = pd.read_csv(IN_FILE, header=0, index_col=None, sep=',')

# 基礎分析

In [3]:
df.shape

(3395, 20)

In [4]:
df.head()

,GameID,LeagueIndex,Age,HoursPerWeek,TotalHours,APM,SelectByHotkeys,AssignToHotkeys,UniqueHotkeys,MinimapAttacks,MinimapRightClicks,NumberOfPACs,GapBetweenPACs,ActionLatency,ActionsInPAC,TotalMapExplored,WorkersMade,UniqueUnitsMade,ComplexUnitsMade,ComplexAbilitiesUsed
0,52,5,27,10,3000,143.7180,0.003515,0.000220,7,0.000110,0.000392,0.004849,32.6677,40.8673,4.7508,28,0.001397,6,0.0,0.000000
1,55,5,23,10,5000,129.2322,0.003304,0.000259,4,0.000294,0.000432,0.004307,32.9194,42.3454,4.8434,22,0.001194,5,0.0,0.000208
2,56,4,30,10,200,69.9612,0.001101,0.000336,4,0.000294,0.000461,0.002926,44.6475,75.3548,4.0430,22,0.000745,6,0.0,0.000189
3,57,3,19,20,400,107.6016,0.001034,0.000213,1,0.000053,0.000543,0.003783,29.2203,53.7352,4.9155,19,0.000426,7,0.0,0.000384
4,58,3,32,10,500,122.8908,0.001136,0.000327,2,0.000000,0.001329,0.002368,22.6885,62.0813,9.3740,15,0.001174,4,0.0,0.000019


In [5]:
df.describe()

,GameID,LeagueIndex,APM,SelectByHotkeys,AssignToHotkeys,UniqueHotkeys,MinimapAttacks,MinimapRightClicks,NumberOfPACs,GapBetweenPACs,ActionLatency,ActionsInPAC,TotalMapExplored,WorkersMade,UniqueUnitsMade,ComplexUnitsMade,ComplexAbilitiesUsed
count,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000
mean,4805.012371,4.184094,117.046947,0.004299,0.000374,4.364654,0.000098,0.000387,0.003463,40.361562,63.739403,5.272988,22.131664,0.001032,6.534021,0.000059,0.000142
std,2719.944851,1.517327,51.945291,0.005284,0.000225,2.360333,0.000166,0.000377,0.000992,17.153570,19.238869,1.494835,7.431719,0.000519,1.857697,0.000111,0.000265
min,52.000000,1.000000,22.059600,0.000000,0.000000,0.000000,0.000000,0.000000,0.000679,6.666700,24.093600,2.038900,5.000000,0.000077,2.000000,0.000000,0.000000
25%,2464.500000,3.000000,79.900200,0.001258,0.000204,3.000000,0.000000,0.000140,0.002754,28.957750,50.446600,4.272850,17.000000,0.000683,5.000000,0.000000,0.000000
50%,4874.000000,4.000000,108.010200,0.002500,0.000353,4.000000,0.000040,0.000281,0.003395,36.723500,60.931800,5.095500,22.000000,0.000905,6.000000,0.000000,0.000020
75%,7108.500000,5.000000,142.790400,0.005133,0.000499,6.000000,0.000119,0.000514,0.004027,48.290500,73.681300,6.033600,27.000000,0.001259,8.000000,0.000086,0.000181
max,10095.000000,8.000000,389.831400,0.043088,0.001752,10.000000,0.003019,0.004041,0.007971,237.142900,176.372100,18.558100,58.000000,0.005149,13.000000,0.000902,0.003084


In [6]:
(df=='?').sum(axis=0)

GameID                   0
LeagueIndex              0
Age                     55
HoursPerWeek            56
TotalHours              57
APM                      0
SelectByHotkeys          0
AssignToHotkeys          0
UniqueHotkeys            0
MinimapAttacks           0
MinimapRightClicks       0
NumberOfPACs             0
GapBetweenPACs           0
ActionLatency            0
ActionsInPAC             0
TotalMapExplored         0
WorkersMade              0
UniqueUnitsMade          0
ComplexUnitsMade         0
ComplexAbilitiesUsed     0
dtype: int64

In [7]:
# '?'を含む行を削除
df = df[(df=='?').sum(axis=1)==0]

In [8]:
# クラスごとのサンプル数
df.LeagueIndex.value_counts()

4    811
5    804
6    621
3    553
2    347
1    167
7     35
Name: LeagueIndex, dtype: int64

In [9]:
# シャッフルとインデックス更新
df = df.sample(frac=1, random_state=0) # 行シャッフル
df = df.reset_index(drop=True) # インデックスの更新

In [10]:
# データセットの作成
target_col = 'LeagueIndex'
target = 7
del_col = [target_col, 'GameID']
X = df.drop(del_col, axis=1)
y = (df[target_col]==target).astype(np.int32)
print(X.shape)

(3338, 18)


# 異常検出（ホールドアウト法による評価）

In [11]:
# ホールドアウト法による精度評価
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=47)

In [12]:
# 直交表の利用
oa = np.reshape(L32, (32, -1)) # 直交表整形
del_cols = range(X.shape[1], oa.shape[1])
oa = np.delete(oa, del_cols, axis=1) # 不要列を削除
sn_ratios = orthogonal_array_test(oa, X_train, y_train)
sn_averages_pos, sn_averages_neg = orthogonal_array_sign(oa, sn_ratios)
print(sn_averages_pos-sn_averages_neg) # 項目ごとのSN比平均差、プラスなら異常検知に関して有意な項目

[-0.16538425  0.68042977 -0.63886082  0.33270743  0.15920755  0.36333522
 -0.25959958  0.6416313  -0.0337364   0.72364117 -0.47404517 -0.03159916
  0.05898177 -0.10132967  0.37801642 -0.08829773  0.30914577 -0.0114109 ]


In [13]:
# 項目選択の結果をもとに異常検知
sign_cols = np.where(sn_averages_pos-sn_averages_neg>=0, 1, 0) # 直交表のテストをもとに有用な変数を選択
arr_cols = list(itertools.compress(X_train.columns, sign_cols))
X_train_ext = X_train[arr_cols]
X_test_ext = X_test[arr_cols]

# MT法による異常度計算
mts = MTS()
mts.fit(X_train_ext[y_train==0])
anomaly_score = mts.predict(X_test_ext)
(break_even_point, score), (coverage, detection) = mts.break_even_point(anomaly_score, y_test)
anomaly_score_abnormal = anomaly_score[y_test==1]

1.3940976092535917 0.8571428571428571 0.8170955882352942


In [14]:
# 項目選択せずに異常検知
mts = MTS()
mts.fit(X_train[y_train==0])
anomaly_score = mts.predict(X_test)
(break_even_point, score), (coverage, detection) = mts.break_even_point(anomaly_score, y_test)
anomaly_score_abnormal = anomaly_score[y_test==1]

1.1567418357472288 0.7857142857142857 0.7729779411764706


# 異常検出（5分割交差検証による評価）

In [15]:
# 項目選択のないMT法（単なるMahalanobis距離）交差確認
scores = []
kf = KFold(n_splits=5, random_state=0, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(X)):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # 項目選択せずに異常検知
    # →平均0.5867
    mts = MTS()
    mts.fit(X_train[y_train==0])
    anomaly_score = mts.predict(X_test)
    (break_even_point, score), (coverage, detection) = mts.break_even_point(anomaly_score, y_test)
    anomaly_score_abnormal = anomaly_score[y_test==1]
    scores.append(score)
print('average: {}'.format(sum(scores)/len(scores)))

1.0276689526725056 1.0 0.724812030075188
1.1821281595983038 0.875 0.7818181818181819
1.2520418358886756 0.8571428571428571 0.8562783661119516
1.2391851832485645 0.9 0.806697108066971
0.878775863271093 0.7142857142857143 0.5787878787878787
average: 0.8692857142857143


In [16]:
# MT法で交差確認
scores = []
kf = KFold(n_splits=5, random_state=0, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(X)):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # 直交表の利用
    oa = np.reshape(L32, (32, -1)) # 直交表整形
    del_cols = range(X.shape[1], oa.shape[1])
    oa = np.delete(oa, del_cols, axis=1) # 不要列を削除
    sn_ratios = orthogonal_array_test(oa, X_train, y_train)
    sn_averages_pos, sn_averages_neg = orthogonal_array_sign(oa, sn_ratios)
    # print(sn_averages_pos-sn_averages_neg) # 項目ごとのSN比平均差、プラスなら異常検知に関して有意な項目
    
    # 項目選択の結果をもとに異常検知
    sign_cols = np.where(sn_averages_pos-sn_averages_neg>=0, 1, 0) # 直交表のテストをもとに有用な変数を選択
    arr_cols = list(itertools.compress(X_train.columns, sign_cols))
    X_train_ext = X_train[arr_cols]
    X_test_ext = X_test[arr_cols]

    # MT法による異常度計算
    # →平均0.6751
    mts = MTS()
    mts.fit(X_train_ext[y_train==0])
    anomaly_score = mts.predict(X_test_ext)
    (break_even_point, score), (coverage, detection) = mts.break_even_point(anomaly_score, y_test)
    anomaly_score_abnormal = anomaly_score[y_test==1]
    scores.append(score)
print('average: {}'.format(sum(scores)/len(scores)))

1.0879950697690004 1.0 0.7609022556390977
1.6324988353656322 0.875 0.8575757575757575
1.7509597252918705 1.0 0.8956127080181543
2.0141861475619356 0.9 0.8995433789954338
1.1226705703260742 0.8571428571428571 0.7212121212121212
average: 0.9264285714285714


# すべてのサンプルを使って項目選択

In [17]:
# 直交表の利用
oa = np.reshape(L32, (32, -1)) # 直交表整形
del_cols = range(X.shape[1], oa.shape[1])
oa = np.delete(oa, del_cols, axis=1) # 不要列を削除
sn_ratios = orthogonal_array_test(oa, X, y)
sn_averages_pos, sn_averages_neg = orthogonal_array_sign(oa, sn_ratios)
print(sn_averages_pos-sn_averages_neg) # 項目ごとのSN比平均差、プラスなら異常検知に関して有意な項目

# 項目選択の結果を出力
sign_cols = np.where(sn_averages_pos-sn_averages_neg>=0, 1, 0) # 直交表のテストをもとに有用な変数を選択
arr_cols = list(itertools.compress(X.columns, sign_cols))
print(arr_cols)

[-0.22419317  0.87383291 -0.65338249  0.34039767  0.21268755  0.51638279
 -0.26591795  0.9173867  -0.07416662  0.7581117  -0.24484961 -0.0862661
 -0.01262095 -0.21830204  0.14843757 -0.15073762  0.19564539  0.05363951]
['HoursPerWeek', 'APM', 'SelectByHotkeys', 'AssignToHotkeys', 'MinimapAttacks', 'NumberOfPACs', 'WorkersMade', 'ComplexUnitsMade', 'ComplexAbilitiesUsed']
